##### (https://www.schoolofhaskell.com/school/to-infinity-and-beyond/pick-of-the-week/coroutines-for-streaming/part-2-coroutines)
### The Coroutine Monad Transformer

In [12]:
data Coroutine s m r = Coroutine { resume :: m(CoroutineState s m r) }

data CoroutineState s m r
  = Run (s (Coroutine s m r))
  | Done r

This is the free monad transformer in diguise.

In [13]:
import Control.Monad
import Control.Monad.Trans.Class

instance (Functor m, Functor s) => Functor (Coroutine s m) where
  fmap f = Coroutine . fmap go . resume where
    go (Done x) = Done $ f x
    go (Run s) = Run $ fmap (fmap f) s
    
instance (Functor s, Applicative m, Monad m) => Applicative (Coroutine s m) where
  pure x = Coroutine . pure $ Done x
  a <*> b = ap a b --Coroutine $ go <$> resume a <*> resume b where
    --go :: CoroutineState s m (a -> b) -> CoroutineState s m a -> CoroutineState s m b
    {- go (Done ab) (Done a) = Done $ ab a
    go (Run sab) (Done a) = Run $ fmap (fmap ($ a)) sab
    go (Done ab) (Run sa) = Run $ fmap (fmap ab) sa
    go (Run sab) (Run sa) = Run $ ap <$> sab <*> sa
    go' = ap -}

instance (Functor s, Monad m) => Monad (Coroutine s m) where
  return x = Coroutine $ return (Done x)
  -- Coroutine s m a -> (a -> Coroutine s m b) -> Coroutine s m b
  f >>= g = Coroutine $ do
    st <- resume f
    case st of
      Done r -> resume (g r)
      Run s -> return . Run $ fmap (>>= g) s
      
instance Functor s => MonadTrans (Coroutine s) where
  lift = Coroutine . (>>= return . Done)
  
instance Show (Coroutine s m r) where
  show _ = "coroutine"
  
suspend :: (Monad m, Functor s) => s (Coroutine s m r) -> Coroutine s m r
suspend = Coroutine . return . Run

The `s` in this type is called the "suspension functor." When a coroutine suspends itself it presents you with the functor that provides a way to resume it.

In [14]:
newtype PauseF x = PauseF x
instance Functor PauseF where
  fmap f (PauseF x) = PauseF $ f x

type PauseT = Coroutine PauseF

pause :: Monad m => PauseT m ()
pause = suspend . PauseF $ return ()

test :: PauseT IO ()
test = do
  lift $ putStrLn "1"
  pause
  lift $ putStrLn "2"
  pause
  lift $ putStrLn "3"
  
runN :: Monad m => Int -> PauseT m () -> m ()
runN 0 = const $ return ()
runN n = go . resume where
  go co = do
    st <- co
    case st of
      Done _ -> return ()
      Run (PauseF x) -> runN (n-1) x
  
runAll :: Monad m => PauseT m () -> m ()
runAll = go . resume where
  --go :: m (CoroutineState PauseF m ()) -> m (CoroutineState PauseF m ())
  go co = do
    st <- co
    case st of
      Done _ -> return ()
      Run (PauseF x) -> runAll x
      
runAll test

runN 1 test

runN 2 test

runN 100 test

1
2
3

1

1
2

1
2
3

A suspension functor that allows for emitting and inputing a value.

In [15]:
data Interface i o x = Produced o (i -> x)

instance Functor (Interface i o) where
  fmap f (Produced o k) = Produced o (f . k)

Coroutines are either consuming or producing. When producing it should run until it produces a value and then switch to consuming, waiting for a value to be supplied.

In [16]:
type Producing o i = Coroutine (Interface i o)
type Consuming r m i o = i -> Producing o i m r

yield :: Monad m => o -> Producing o i m i
yield x = suspend $ Produced x return


--                           o i                      i o
($$) :: Monad m => Producing a b m r -> Consuming r m a b -> m r
pro $$ co = go $ resume pro where
  go mpst = do
    pst <- mpst
    case pst of
      Done x -> return x
      Run (Produced x cb1) -> co x $$ cb1
      
prod :: Producing String String IO String
prod = do
  lift $ putStrLn "a1"
  x <- yield "hello"
  lift $ putStrLn x
  pure ""
  
consume :: Consuming String IO String String
consume i = do
  lift $ putStrLn i
  yield "test"
  pure ""
  
prod $$ consume

a1
hello
test
""

Specialize `Coroutine` by having `Interface` be built in and defining Producing and Consuming as newtypes.

In [17]:
newtype Producing o i m r = Producing { resume :: m ( ProducerState o i m r ) }

data ProducerState o i m r
  = Done r
  | Produced o (Consuming r m i o)
  
newtype Consuming r m i o = Consuming { provide :: i -> Producing o i m r }

instance Functor m => Functor (Producing o i m) where
  fmap f = Producing . fmap (fmap f) . resume

instance Functor m => Functor (ProducerState o i m) where
  fmap f (Done r) = Done $ f r
  fmap f (Produced o c) = Produced o . Consuming $ (fmap f . provide c)
  
instance Monad m => Applicative (Producing o i m) where
  pure = Producing . return . Done
  (<*>) = ap
  
instance Monad m => Monad (Producing o i m) where
  return = pure
  a >>= rb = Producing $ (resume a) >>= go where
    --go :: ProducerState o i m r -> m (ProducerState o i m b)
    go (Done r) = resume $ rb r
    go (Produced o c) = return $ Produced o (Consuming . fmap (>>= rb) $ provide c)
    
instance MonadTrans (Producing o i) where
  lift = Producing . liftM Done
  
yield :: Monad m => o -> Producing o i m i
yield x = Producing . return $ Produced x (Consuming return)

infixl 0 $$
($$) :: Monad m => Producing o i m r -> Consuming r m o i -> m r
p $$ c = resume p >>= go where
  go (Done r) = return r
  go (Produced o pc) = provide c o $$ pc
  
  
example1 :: Producing String String IO ()
example1 = do
  name <- yield "What's your name?"
  lift $ putStrLn $ "Hello " ++ name ++ "!"
  color <- yield "Favorite color?"
  lift $ putStrLn $ color ++ " is a terrible color. You must repent."
  
foreverK :: Monad m => (a -> m a) -> a -> m r
foreverK f = go where
  go a = f a >>= go
  
stdOutIn :: Consuming r IO String String
stdOutIn = Consuming . foreverK $ \str -> do
  lift $ putStrLn str
  yield "steve"
  --lift getLine >>= yield
  
stdInOut :: Producing String String IO r
stdInOut = provide stdOutIn ""
  
example1 $$ stdOutIn

What's your name?
Hello steve!
Favorite color?
steve is a terrible color. You must repent.

### Proxies

What does the type `Producing a b (Producing c d) m` do?
It's a computation that can transfer control to one of two interfaces. `yield a` will surrender control to the outer interface and `lift (yield a)` will transfer control to the inner interface.

`p :: Producing a b (Producing c d m) r
c :: Consuming r (Producing c d m) a b
p $$ c :: Producing c d m r`

In order to connect a `Producing a b m r` to `Consuming r (Producing c d m) a b` we can use `lift` and `hoist`

In [18]:
:ext RankNTypes


{-
newtype Producing o i m r = Producing { resume :: m ( ProducerState o i m r ) }

data ProducerState o i m r
  = Done r
  | Produced o (Consuming r m i o)
  
newtype Consuming r m i o = Consuming { provide :: i -> Producing o i m r }
-}


--Lift a monad morphism from m to n into a monad morphism from (t m) to (t n)
--The first argument to hoist must be a monad morphism, even though the type system does not enforce this

hoist :: Monad m => (forall a. m a -> n a) -> Producing o i m b -> Producing o i n b
hoist f = go where
  go p = Producing $ f $ liftM map' (resume p)
  map' (Done x) = Done x
  map' (Produced o c) = Produced o $ Consuming (go . provide c)
  
insert0 = lift
insert1 = hoist insert0
insert2 = hoist insert1

p :: Producing Int Int IO ()
p = Producing $ fmap Done ( putStrLn "Hello" )

c :: Consuming () (Producing String String IO) Int Int
c = Consuming $ \i -> Producing (Producing $ putStrLn (show i) >> return (Done (Done ())))

resume (insert1 p $$ c)

Hello

We have stumbled on the idea of a `Proxy`

In [19]:
:ext TypeFamilies
:ext DataKinds

type family Fst (xy :: (*,*)) :: * where
  Fst '(x, _) = x
  
type family Snd (xy :: (*, *)) :: * where
  Snd '(_, y) = y

newtype Proxy r m upI downI
  = Proxy { unProxy :: Consuming r (Producing (Fst downI) (Snd downI) m) (Fst upI) (Snd upI) }
  
($=) :: Monad m => Producing a b m r -> Proxy r m '(a,b) '(c,d) -> Producing c d m r
prod $= proxy = insert1 prod $$ unProxy proxy

Proxys have a downstream interface and an upstream interface. We can connect a proxy to a `Producing` Coroutine via the upstream interface (the outer coroutine) or to a `Consuming` Coroutine via the downstream interface (the inner Coroutine).

The input and output types of these interfaces are expressed as a type level tuples so that we can write a Category instance for Proxy.

## Producing Layers Commute

`Producing a b (Producing c d m) r` is the same thing as `Producing c d (Producing a b m) r`.

In [95]:
:ext ScopedTypeVariables

yield :: Monad m => o -> Producing o i m i
yield x = Producing . return $ Produced x (Consuming return)

-- The proxy newtype was left out for simplicity
idProxy :: Monad m => Consuming r (Producing a b m) a b
idProxy = Consuming $ \a -> 
  Producing (Producing 
  $ return (Produced a 
  (Consuming $ \b -> Producing 
  $ return (Done (Produced b idProxy)))))
-- bonus: implement this by also assuming yield as a given
idProxy' = Proxy $ Consuming $ foreverK $ lift . yield >=> yield

-- show implement this (the puzzle)
commute :: forall a b c d m r. Monad m =>
  Producing a b (Producing c d m) r -> Producing c d (Producing a b m) r
commute p = p' $$ funnel where
  -- what types should p' and funnel have? (leverage scoped type variables)
  p' :: Producing a b (Producing c d (Producing a b m)) r
  p' = insert2 p
  funnel :: Consuming r (Producing c d (Producing a b m)) a b
  funnel = Consuming $ insert1 . provide idProxy
  -- types hint: remember, $$ removes the outermost interface
  -- implementation hint: use insert0/1/2 with p and idProxy
  
  
-- show implement these
(=$) :: forall a b c d m r. Monad m =>
  Proxy r m '(a,b) '(c,d) -> Consuming r m c d -> Consuming r m a b
Proxy proxy =$ consuming = Consuming $ \(a :: a) ->
  let
    p :: Producing c d (Producing b a m) r
    p = commute $ provide proxy a
    c :: Consuming r (Producing b a m) c d
    c = Consuming $ \c -> insert1 $ provide consuming c 
  in
    p $$ c :: Producing b a m r

(=$=) :: forall a a' b b' c c' m r. Monad m =>
  Proxy r m '(a,a') '(b,b') -> Proxy r m '(b,b') '(c,c') -> Proxy r m '(a,a') '(c,c')
Proxy proxyl =$= Proxy proxyr = Proxy $ Consuming $ \(a :: a) ->
  let
    p :: Producing b b' (Producing a' a (Producing c c' m)) r
    p = insert2 . commute $ provide proxyl a
    c :: Consuming r (Producing a' a (Producing c c' m)) b b'
    c = Consuming (\b -> insert1 (provide proxyr b))
  in
    p $$ c :: Producing a' a (Producing c c' m) r
    